In [1]:
import numpy as np
import argparse
import random
import shutil
import time
import warnings

from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn

import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim as optim

import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed

import torchvision
from torchvision import datasets, models, transforms
import time
import os
import sys
import copy

In [ ]:
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
# import torchvision.models as models

In [2]:
torch.cuda.device_count()

2

In [18]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [19]:
# See how many devices are around
torch.cuda.device_count()
# Set it to a particular device
torch.cuda.set_device(1)
# Check which device you are on
torch.cuda.current_device()

1

In [6]:
save_path='/scratch/by783/DL_Final_models/'+'190424_vgg_ae'#args.save
model_name = 'vgg'#args.model
num_epochs = 10 #args.epochs
feature_extract = True # str2bool(args.pretrained)

###################### fixed_params ###################################

num_classes = 1000
loader_image_path='/scratch/by783/DL_Final/ssl_data_96'
loader_batch_size=256

In [7]:
def image_loader(path, batch_size):
    transform = transforms.Compose(
        [
            #transforms.Resize(input_size),
            #transforms.CenterCrop(input_size),
            # use model fitted with the image size, so no need to resize
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            # https://pytorch.org/docs/stable/torchvision/transforms.html
            # [mean],[std] for different channels
        ]
    )
    sup_train_data = datasets.ImageFolder('{}/{}/train'.format(path, 'supervised'), transform=transform)
    sup_val_data = datasets.ImageFolder('{}/{}/val'.format(path, 'supervised'), transform=transform)
    unsup_data = datasets.ImageFolder('{}/{}/'.format(path, 'unsupervised'), transform=transform)
    # source code: https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py
    # Main idea:
    data_loader_sup_train = torch.utils.data.DataLoader(
        sup_train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    data_loader_sup_val = torch.utils.data.DataLoader(
        sup_val_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    data_loader_unsup = torch.utils.data.DataLoader(
        unsup_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )

    print('sup_train_data.class_to_idx==sup_val_data.class_to_idx: ',
          sup_train_data.class_to_idx == sup_val_data.class_to_idx)

    return data_loader_sup_train, data_loader_sup_val, data_loader_unsup, sup_train_data.class_to_idx

In [8]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False




def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0



    if model_name != "vgg":
        sys.stdout.write('We only have vgg now!!!')
    else:
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)

        ##########

        model_ft.avgpool = nn.AdaptiveAvgPool2d(output_size=(3, 3))

        model_ft.classifier[0] = nn.Linear(in_features=4608, out_features=4096, bias=True)
        model_ft.classifier[3] = nn.Linear(in_features=4096, out_features=4096, bias=True)
        model_ft.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes, bias=True)


        input_size = 96

    return model_ft, input_size

In [11]:
use_pretrained=True
model_ft = models.vgg11_bn(pretrained=use_pretrained)
set_parameter_requires_grad(model_ft, feature_extract)

In [13]:
model_ft.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [15]:
class ConvolutionalAutoencoder(torch.nn.Module):

    def __init__(self):
        super(ConvolutionalAutoencoder, self).__init__()
        
        # calculate same padding:
        # (w - k + 2*p)/s + 1 = o
        # => p = (s(o-1) - w + k)/2
        
        ### ENCODER
        
        # 28x28x1 => 28x28x4
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                      out_channels=4,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      # (1(28-1) - 28 + 3) / 2 = 1
                                      padding=1) 
        # 28x28x4 => 14x14x4                              
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         # (2(14-1) - 28 + 2) / 2 = 0
                                         padding=0)                                       
        # 14x14x4 => 14x14x8
        self.conv_2 = torch.nn.Conv2d(in_channels=4,
                                      out_channels=8,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      # (1(14-1) - 14 + 3) / 2 = 1
                                      padding=1)                 
        # 14x14x8 => 7x7x8                             
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         # (2(7-1) - 14 + 2) / 2 = 0
                                         padding=0)
        
        ### DECODER
                                         
        # 7x7x8 => 15x15x4                          
        self.deconv_1 = torch.nn.ConvTranspose2d(in_channels=8,
                                                 out_channels=4,
                                                 kernel_size=(3, 3),
                                                 stride=(2, 2),
                                                 padding=0)
        
        # 15x15x4  => 31x31x1                           
        self.deconv_2 = torch.nn.ConvTranspose2d(in_channels=4,
                                                 out_channels=1,
                                                 kernel_size=(3, 3),
                                                 stride=(2, 2),
                                                 padding=0)
        
    def forward(self, x):
        
        ### ENCODER
        x = self.conv_1(x)
        x = F.leaky_relu(x)
        x = self.pool_1(x)
        x = self.conv_2(x)
        x = F.leaky_relu(x)
        x = self.pool_2(x)
        
        ### DECODER
        x = self.deconv_1(x)
        x = F.leaky_relu(x)
        x = self.deconv_2(x)
        x = F.leaky_relu(x)
        logits = x[:, :, 2:30, 2:30]
        probas = torch.sigmoid(logits)
        return logits, probas

    
torch.manual_seed(1)
model_try = ConvolutionalAutoencoder()

In [16]:
model_try

ConvolutionalAutoencoder(
  (conv_1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool_1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool_2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (deconv_1): ConvTranspose2d(8, 4, kernel_size=(3, 3), stride=(2, 2))
  (deconv_2): ConvTranspose2d(4, 1, kernel_size=(3, 3), stride=(2, 2))
)

In [18]:
model_ft.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU(inplace)
  (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU(inplace)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (15)

In [22]:
# https://stackoverflow.com/questions/37837682/python-class-input-argument/37837766
# https://github.com/awentzonline/pytorch-cns/blob/master/examples/vggmse.py

class Model_Based_Autoencoder(torch.nn.Module):
    def __init__(self,model_name):
        super(Model_Based_Autoencoder, self).__init__()
        if model_name!='vgg':
            sys.stdout.write('Dear, we only support vgg now...')
        
        self.encoder = models.vgg11_bn(pretrained=use_pretrained).features
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512,512,kernel_size=(2, 2), stride=(0, 0), padding=(2, 2)),#de-conv8
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512,512,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),#de-conv7
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512,512,kernel_size=(2, 2), stride=(0, 0), padding=(2, 2)),#de-conv6
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512,256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),#de-conv5
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256,256,kernel_size=(2, 2), stride=(0, 0), padding=(2, 2)),#de-conv4
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256,128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),#de-conv3
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128,64,kernel_size=(2, 2), stride=(0, 0), padding=(2, 2)),#de-conv2
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64,3,kernel_size=(2, 2), stride=(0, 0), padding=(2, 2)),#de-conv1
            nn.BatchNorm2d(3),
            nn.Tanh()
        )
    
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
            
    

In [32]:
model_vggae=Model_Based_Autoencoder('vgg')

NameError: name 'Model_Based_Autoencoder' is not defined

In [32]:
set_parameter_requires_grad(model_vggae.encoder, feature_extract)

In [33]:
model_vggae.encoder[0].weight.requires_grad

False

In [35]:
model_vggae.decoder[0].weight.requires_grad

True

In [37]:
criterion = nn.MSELoss()

learning_rate=0.001
optimizer = torch.optim.Adam(model_vggae.parameters(), lr=learning_rate, weight_decay=1e-5)



In [ ]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './dc_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './conv_autoencoder.pth')

In [29]:
torch.tensor([0]).item()

0

In [30]:
(1,0)[0]

1

In [31]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    #unsupervised learning, we do not need train and vals
    since = time.time()
    loss_history=[]
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    
    for epoch in range(num_epoches):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        ################# train the model on unsupervised data ############
        model.train()
        
        running_loss = 0.0
        
        for inputs, _ in dataloaders['unlabeled']:
            inputs = inputs.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            
            running_loss += loss.item() * inputs.size(0)
            
            loss.backward()
            optimizer.step()
            
        epoch_loss = running_loss / len(dataloaders['unlabeled'].dataset)
        sys.stdout.write('Training time: {:.0f}s'.format( time.time() - since ))
        
        ################# evaluate the model performance on labeled data ############
        
        model.eval()
        
        eval_loss=0.0
        for inputs, _ in dataloaders['labeled']:
            inputs = inputs.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            eval_loss += loss.item() * inputs.size(0)
            
        epoch_eval_loss = running_loss / len(dataloaders['labeled'].dataset)
        sys.stdout.write('Evaluation time: {:.0f}s'.format( time.time() - since ))        
        sys.stdout.write('Training loss: {:.4f} Eval loss: {:.4f}'.format(epoch_loss, epoch_eval_loss))
        
        #################
        
        loss_history.append( ( epoch_loss.item(),epoch_eval_loss.item() ) )
        
        
        if epoch_eval_loss < best_loss:
            best_loss = epoch_eval_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            with open(save_path, 'wb') as f:
                torch.save(model, f)
            
        with open(save_path+'_val_acc', 'w') as f:
            for item in loss_history:
                f.write("unlabeled: %s, labeled: s% \n,  " % (item[0],item[1]) )
    
    
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return model, loss_history
        